In [1]:
function f(x_prev, v, dt)
    x_prev + v*dt
end

@code_native debuginfo=:none dump_module=false f(1.0, 2.0, 0.1)

	.text
	push	rbp
	mov	rbp, rsp
	mov	rax, qword ptr [r13 + 16]
	mov	rax, qword ptr [rax + 16]
	mov	rax, qword ptr [rax]
	vmulsd	xmm1, xmm1, xmm2
	vaddsd	xmm0, xmm1, xmm0
	pop	rbp
	ret
	nop	dword ptr [rax]


In [2]:
@fastmath function f_fastmath(x_prev, v, dt)
    x_prev + v*dt
end

@code_native debuginfo=:none dump_module=false f_fastmath(1.0, 2.0, 0.1)

	.text
	push	rbp
	mov	rbp, rsp
	mov	rax, qword ptr [r13 + 16]
	mov	rax, qword ptr [rax + 16]
	mov	rax, qword ptr [rax]
	vfmadd231sd	xmm0, xmm2, xmm1        # xmm0 = (xmm2 * xmm1) + xmm0
	pop	rbp
	ret
	nop	word ptr cs:[rax + rax]


In [3]:
"""
Solve x(0) = 0, x'(t) = x(t) + 1, 0 ≤ t ≤ 1.

Exact solution: x(t) = exp(t) - 1, x(1) = exp(1) - 1 = 1.718281828⋯.
"""
function solveode(f, n=10^6, x0=0.0, dt=1/n)
    x = x0
    for _ in 1:n
        x = f(x, x+1, dt)
    end
    x
end

@show solveode(f)
@show solveode(f_fastmath)
@show exp(1) - 1;

solveode(f) = 1.7182804693193672
solveode(f_fastmath) = 1.7182804693193663
exp(1) - 1 = 1.718281828459045


In [4]:
using BenchmarkTools

@btime solveode(f)
@btime solveode(f_fastmath);

  2.627 ms (0 allocations: 0 bytes)
  54.400 μs (0 allocations: 0 bytes)


In [5]:
@code_native debuginfo=:none dump_module=false solveode(f)

	.text
	push	rbp
	mov	rbp, rsp
	mov	rax, qword ptr [r13 + 16]
	mov	rax, qword ptr [rax + 16]
	mov	rax, qword ptr [rax]
	vxorpd	xmm0, xmm0, xmm0
	mov	eax, 1000000
	movabs	rcx, offset .rodata.cst8
	vmovsd	xmm1, qword ptr [rcx]           # xmm1 = mem[0],zero
	movabs	rcx, 2686337197992
	vmovsd	xmm2, qword ptr [rcx]           # xmm2 = mem[0],zero
	nop	word ptr cs:[rax + rax]
L64:
	vaddsd	xmm3, xmm0, xmm1
	vmulsd	xmm3, xmm3, xmm2
	vaddsd	xmm0, xmm0, xmm3
	vaddsd	xmm3, xmm0, xmm1
	vmulsd	xmm3, xmm3, xmm2
	vaddsd	xmm0, xmm0, xmm3
	vaddsd	xmm3, xmm0, xmm1
	vmulsd	xmm3, xmm3, xmm2
	vaddsd	xmm0, xmm0, xmm3
	vaddsd	xmm3, xmm0, xmm1
	vmulsd	xmm3, xmm3, xmm2
	vaddsd	xmm0, xmm0, xmm3
	vaddsd	xmm3, xmm0, xmm1
	vmulsd	xmm3, xmm3, xmm2
	vaddsd	xmm0, xmm0, xmm3
	vaddsd	xmm3, xmm0, xmm1
	vmulsd	xmm3, xmm3, xmm2
	vaddsd	xmm0, xmm0, xmm3
	vaddsd	xmm3, xmm0, xmm1
	vmulsd	xmm3, xmm3, xmm2
	vaddsd	xmm0, xmm0, xmm3
	vaddsd	xmm3, xmm0, xmm1
	vmulsd	xmm3, xmm3, xmm2
	vaddsd	xmm0, xmm0, xmm3
	vaddsd	xmm3, xmm0, xm

In [6]:
@code_native debuginfo=:none dump_module=false solveode(f_fastmath)

	.text
	push	rbp
	mov	rbp, rsp
	mov	rax, qword ptr [r13 + 16]
	mov	rax, qword ptr [rax + 16]
	mov	rax, qword ptr [rax]
	vxorpd	xmm0, xmm0, xmm0
	mov	eax, 1000000
	movabs	rcx, offset .rodata.cst8
	vmovsd	xmm1, qword ptr [rcx]           # xmm1 = mem[0],zero
	movabs	rcx, 2686337198120
	vmovsd	xmm2, qword ptr [rcx]           # xmm2 = mem[0],zero
	nop	word ptr cs:[rax + rax]
L64:
	vfmadd213sd	xmm0, xmm1, xmm2        # xmm0 = (xmm1 * xmm0) + xmm2
	add	rax, -16
	jne	L64
	pop	rbp
	ret
	nop	dword ptr [rax]
